# جهت اجرای نوت‌بوک لطفا فایل زیر را دانلود کنید و در فولدر مربوط به همین فایل قرار دهید.


https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fa.vec

### IMPORT LIBRARIES

In [1]:
import pandas as pd
import codecs
import fasttext
from hazm import *
import re
import random
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import wordnet
import nltk
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
from math import log2

### LOAD DATASET

In [2]:
df = pd.read_csv('pre-proc-data2.csv')
text = df['comment']
df.sentiment.value_counts()

-1    2975
 1    2354
 0    1879
Name: sentiment, dtype: int64

### Preparing the dataset to be compatible with the fasttext structure.

In [3]:
df = df.replace(-1 , 'neg')

In [4]:
df['sentiment'] = "__label__" + df['sentiment'].astype(str)
df.head(3)

,Unnamed: 0,comment,sentiment,num_tok
0,0,آقای رشیدپور من تا هفته NUM۲ بچه‌ام پسر بود و ...,__label__1,31
1,1,چقدر این دزد باحال بود,__label__1,5
2,2,سلام اگر ممکنه از کلمه سی تی اسکن تو برنامتون ...,__label__neg,19


In [5]:
df['sentiment_comment'] = df['sentiment'] +' '+ df['comment']
df.head(3)

,Unnamed: 0,comment,sentiment,num_tok,sentiment_comment
0,0,آقای رشیدپور من تا هفته NUM۲ بچه‌ام پسر بود و ...,__label__1,31,__label__1 آقای رشیدپور من تا هفته NUM۲ بچه‌ام...
1,1,چقدر این دزد باحال بود,__label__1,5,__label__1 چقدر این دزد باحال بود
2,2,سلام اگر ممکنه از کلمه سی تی اسکن تو برنامتون ...,__label__neg,19,__label__neg سلام اگر ممکنه از کلمه سی تی اسکن...


In [6]:
def pre(text):
    text = re.sub(r'[^\w\s]',' ', text)
    return text

In [7]:
df['sentiment_comment'] = [pre(str(i)) for i in df['sentiment_comment']]

### split dataset to train, test and valid sets.

In [8]:
train, test = train_test_split(df, test_size=0.1, shuffle=True)
train , valid = train_test_split(train, test_size=0.1, shuffle=True)

In [9]:
train.to_csv("t.train", columns= ['sentiment_comment'], index = False, header = False)
test.to_csv("t.test", columns= ['sentiment_comment'], index = False, header = False)
valid.to_csv("t.valid", columns= ['sentiment_comment'], index = False, header = False)

### train a model using fasttext 

In [10]:
model = fasttext.train_supervised("t.train", dim=300, autotuneValidationFile='t.valid', pretrainedVectors='wiki.fa.vec')

Progress: 100.0% Trials:   10 Best score:  0.764253 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  14904
Number of labels: 3
Progress: 100.0% words/sec/thread:  103793 lr:  0.000000 avg.loss:  0.417233 ETA:   0h 0m 0s


In [11]:
model.test('t.test')

(721, 0.7253814147018031, 0.7253814147018031)

### Accuracy without Data Augmentation

In [12]:
acc = 0
for i in range(np.array(test).shape[0]):
    if model.predict(np.array(test['comment'])[i])[0][0] == np.array(test['sentiment'])[i] : 
        acc += 1
print(f'accuracy is {acc/np.array(test).shape[0] *100}%')
    

accuracy is 72.5381414701803%


### -------

In [13]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()
tagger = POSTagger(model='resources-0/postagger.model')

In [14]:
def same_root(word1, word2, n=3):
    # Extract n-grams of the words
    word1_ngrams = set(ngrams(word1, n, pad_left=True, pad_right=True))
    word2_ngrams = set(ngrams(word2, n, pad_left=True, pad_right=True))

    # Compare the n-grams of the words
    common_ngrams = word1_ngrams & word2_ngrams
    return len(common_ngrams) / len(word1_ngrams | word2_ngrams)

# Test the function
print(same_root("نبودم", "بود"))

0.09090909090909091


In [15]:
def rec_diff(word1, word2):
    score = same_root(word1, word2)
    if score >= 0.2:
        return False
    return True

In [16]:
rec_diff('بودیییییییییییییییی','سلام!')

True

In [17]:
vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,1), analyzer='word')
vectorizer_1 = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,1), analyzer='word')
vectorizer_neg = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,1), analyzer='word')
vectorizer_0 = TfidfVectorizer(use_idf = True, norm ='l2', ngram_range=(1,1), analyzer='word')

In [18]:
cond_posetive = train['sentiment'] == '__label__1'
cond_negative = train['sentiment'] == '__label__neg'
cond_nutral = train['sentiment'] == '__label__0'
train_1 = train.loc[cond_posetive, :]
train_neg = train.loc[cond_negative, :]
train_0 = train.loc[cond_nutral, :]
doc_term_mat = vectorizer.fit_transform([doc for doc in train['sentiment_comment']])
doc_term_mat_1 = vectorizer_1.fit_transform([doc for doc in train_1['sentiment_comment']])
doc_term_mat_neg = vectorizer_neg.fit_transform([doc for doc in train_neg['sentiment_comment']])
doc_term_mat_0 = vectorizer_0.fit_transform([doc for doc in train_0['sentiment_comment']])
words = vectorizer.get_feature_names_out()
words_1 = vectorizer_1.get_feature_names_out()
words_0 = vectorizer_0.get_feature_names_out()
words_neg = vectorizer_neg.get_feature_names_out()
doc_term_mat_1_sim = doc_term_mat_1.T.dot(doc_term_mat_1)
doc_term_mat_neg_sim = doc_term_mat_neg.T.dot(doc_term_mat_neg)
doc_term_mat_0_sim = doc_term_mat_0.T.dot(doc_term_mat_0)
doc_term_mat_sim = doc_term_mat.T.dot(doc_term_mat)

In [19]:
def most_similar_word(word, word_list, doc_term, vectorizer):
    word_index = np.where(word_list == word)[0][0]
    word_row = doc_term.T[word_index, :]
    similarities = cosine_similarity(word_row, doc_term.T)
    most_similar_index = similarities.argsort()[0][-10:]
    most_similar_word = vectorizer.get_feature_names_out()[most_similar_index]
    return  most_similar_word

In [20]:
word = 'سلام'
most_similar_word(word, words, doc_term_mat, vectorizer)

array(['خسته', 'آقای', 'برنامه', 'از', '__label__1', 'رشید', 'پور', 'صبح',
       'بخیر', 'سلام'], dtype=object)

In [21]:
feature_names = vectorizer.get_feature_names_out()
idf = vectorizer.idf_

# Create a dictionary of words and their IDF values
word_idf = dict(zip(feature_names, idf))

# Sort the words by their IDF values
sorted_words = sorted(word_idf.items(), key=lambda x: x[1])

# Define a threshold for the IDF values to consider a word as stop word
threshold = 0.5

# Print the words with an IDF value less than the threshold
stop_words = [word for word, idf in sorted_words if idf < 5]
stop_words.append('و')
stop_words.append('ی')
stop_words.append('م')

print(stop_words)

['__label__neg', '__label__1', '__label__0', 'به', 'از', 'که', 'رشید', 'پور', 'این', 'سلام', 'با', 'رو', 'برنامه', 'آقای', 'اقای', 'در', 'شما', 'هم', 'من', 'خیلی', 'ما', 'تو', 'رشیدپور', 'مردم', 'کنید', 'چرا', 'همه', 'را', 'برای', 'خدا', 'لطفا', 'بود', 'های', 'حالا', 'link', 'ها', 'یه', 'میشه', 'تا', 'ولی', 'فقط', 'بی', 'یک', 'عالی', 'می', 'واقعا', 'چه', 'نیست', 'ای', 'عزیز', 'دیگه', 'باید', 'خورشید', 'سال', 'num۲', 'هر', 'شده', 'روز', 'اون', 'جناب', 'داره', 'num۱', 'نه', 'حرف', 'باشه', 'ممنون', 'ایران', 'هست', 'خوب', 'دارم', 'صبح', 'یا', 'خسته', 'شد', 'میکنم', 'کار', 'برنامتون', 'است', 'رضا', 'کردن', 'باشید', 'هیچ', 'اگه', 'بشه', 'دست', 'عالیه', 'چی', 'همیشه', 'بر', 'کنه', 'داریم', 'صحبت', 'امروز', 'مثل', 'کی', 'بعد', 'پایتخت', 'چون', 'الان', 'دوست', 'اگر', 'تمام', 'تون', 'هستیم', 'نباشید', 'هستم', 'کم', 'برف', 'و', 'ی', 'م']


### Unconditional Data Augmentation

In [22]:
# similarity dictionary :)
sim_dict = {}
for word in tqdm.tqdm(words):
    sim_dict[word] = most_similar_word(word, words, doc_term_mat, vectorizer)

100%|████████████████████████████████████| 14863/14863 [01:41<00:00, 146.71it/s]


In [23]:
# FIND THE BEST CANDIDATE OF CHANGEBALE WORDS IN A SENTENCE 
def change_word(text):
    global sim_dict
    tokens = word_tokenize(text)
    sim_vec = []
    for i in tokens:
        flag = True
        if i not in stop_words and i in sim_dict.keys():
            for k in range(7):
                if flag:
                    try: 
                        sim_word = sim_dict[i][k]
                        temp = sim_word
                        #print(temp)
                        if rec_diff(i, temp) and tagger.tag(word_tokenize(temp))[0][1] == tagger.tag(word_tokenize(i))[0][1]:
                            flag = False
                            sim_vec.append(temp)
                                #print(sim_vec)
                        elif k==9:
                                sim_vec.append(' ')
                    except IndexError :
                        #print(tagger.tag(word_tokenize(temp)))
                        pass
                        #sim_vec.append(' ')
                        
        else:
            sim_vec.append(' ')
    
    
    return sim_vec
    

In [24]:
from copy import deepcopy
def generate_sentence_permutations(words1, words2):
    permutations = []
    permutations.append(' '.join(words2))
    if len(words1) != len(words2):
        return permutations
    for i in range(len(words1)):
        
        if not words1[i] == ' ':
            words2[i] = words1[i]
            words1[i] = ' '
        else:
            continue
    temp = ' '.join(words2).strip()
    if not temp == permutations[0]:
        permutations.append(temp)

    return permutations


In [25]:
augmented_list = []
for i in tqdm.tqdm((train['sentiment_comment'])):
    comment = i
    candidate = change_word(comment)
    augmented_list.append(generate_sentence_permutations(candidate, word_tokenize(comment)))
    

100%|█████████████████████████████████████| 5838/5838 [00:05<00:00, 1065.65it/s]


In [26]:

# Initialize an empty list to hold the 1D list
one_d_list = []

# Use nested for loops to iterate over the sublists and items
for sublist in augmented_list:
    for item in sublist:
        one_d_list.append(item)

print(len(one_d_list))


7306


In [27]:
df_nonconditional = pd.DataFrame(one_d_list)
df_nonconditional.rename(columns = { 0 :'sentiment_comment', 1: 'numb' }, inplace = True)
df_nonconditional[df_nonconditional['sentiment_comment'].str.strip().astype(bool)]

,sentiment_comment
0,__label__neg از محمد اصفهانی دعوت نکنین
1,__label__neg از ونوید نکنین خوانندهدها قضانمیشه
2,__label__1 سلام با تشکر فراوان از برنامه پر ان...
3,__label__neg رشیدپور از چشم همه افتاده نون به ...
4,__label__0 سالار عقیلی شماره ۶
...,...
7301,__label__neg مسوولان در حال شمردن پولاشونن وقت...
7302,__label__neg چطوری سوپاپ اطمینان کی میخوای خجا...
7303,__label__0 به آقا تنابنده بگید یه تیکه شمالی ح...
7304,__label__0 به نگار ازآقای بسازند یه سانسور جدی...


In [28]:
df_nonconditional['sentiment_comment'].to_csv('Uncond_aug_TFIDF.train', index=False)

In [29]:
model2 = fasttext.train_supervised("Uncond_aug_TFIDF.train", dim=300, autotuneValidationFile='t.valid', pretrainedVectors='wiki.fa.vec')

Progress: 100.0% Trials:   11 Best score:  0.767334 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  14908
Number of labels: 3
Progress: 100.0% words/sec/thread:  146940 lr:  0.000000 avg.loss:  0.506957 ETA:   0h 0m 0s


In [30]:
model2.test('t.test')

(721, 0.723994452149792, 0.723994452149792)

In [31]:
acc = 0
for i in range(np.array(test).shape[0]):
    if model2.predict(np.array(test['comment'])[i])[0][0] == np.array(test['sentiment'])[i] : 
        acc += 1
print(f'accuracy is {acc/np.array(test).shape[0] *100}%')
    

accuracy is 73.09292649098474%


### perpelxity 

In [32]:
model2.predict('قسمت سانسور شده چیا بودن', k=4)[1]

array([0.87191868, 0.12578608, 0.00232524])

In [33]:
result = [list_[1] for list_ in augmented_list if len(list_) ==2]


In [34]:
generated_sentences_uncond = [' '.join(word_tokenize(sent)[1:]) for sent in result]

In [35]:
n = sum(len(sentence.split()) for sentence in generated_sentences_uncond)
log_probs = [np.log2(model2.predict(sentence, k=1)[1]) for sentence in generated_sentences_uncond]
perplexity = 2 ** (-1/n * sum(log_probs))
print("Perplexity:", perplexity)

Perplexity: [1.03191698]


### diversity 

In [36]:
vectors_uncond = [model2.get_sentence_vector(sentence) for sentence in generated_sentences_uncond]
similarities = cosine_similarity(vectors_uncond)
diversity = 1 - similarities.mean()
print("Diversity:", diversity)

Diversity: 0.6769970655441284


### ارزیابی انسانی

In [54]:
res = []
res = [list_ for list_ in augmented_list if len(list_) ==2]
for i in range(10):
    print(f'org{i+1} : {res[i][0]} \n gen{i+1} : {res[i][1]} \n')

org1 : __label__neg از محمد اصفهانی دعوت نکنین 
 gen1 : __label__neg از ونوید نکنین خوانندهدها قضانمیشه 

org2 : __label__0 سالار عقیلی شماره ۶ 
 gen2 : __label__0 چاووشی اسطوره smsدادم ۶ 

org3 : __label__1 یزد خودونم برف أومده إقه خش شده 
 gen3 : __label__1 امیر برف برف خودشونن خودشونن خودشونن شده 

org4 : __label__0 مناظره بین دکتر سبطی وآقای بطحایی 
 gen4 : __label__0 ودکتر ودکتر بدقولی آقائی نشاط ازقول 

org5 : __label__neg برنامه تون و تمام نکنید لطفا 
 gen5 : __label__neg برنامه تون و تمام وجدانند لطفا 

org6 : __label__0 قسمت سانسور شده چیا بودن 
 gen6 : __label__0 کامیون تیکه شده خودشن همسرشون 

org7 : __label__0 همه بازماندگانی که سال NUM۲ سلامت به پایان رساندن 
 gen7 : __label__0 همه سال که سال NUM۲ درصدف به انتظارمان سال 

org8 : __label__0 محسن از مبا 
 gen8 : __label__0 شحریان از خودشون 

org9 : __label__0 استاد امیر بیات 
 gen9 : __label__0 عاشقتیم دهنت خودشمااقای 

org10 : __label__0 مناظره باوزیر آموزش وپرورش 
 gen10 : __label__0 ودکتر خودشونو دارایی خودشونو 



### Conditional Data Augmentation

In [37]:
sim_dict_1 = {}
for word in tqdm.tqdm(words_1):
    sim_dict_1[word] = most_similar_word(word, words_1, doc_term_mat_1, vectorizer_1)
    
sim_dict_0 = {}
for word in tqdm.tqdm(words_0):
    sim_dict_0[word] = most_similar_word(word, words_0, doc_term_mat_0, vectorizer_0)
    
sim_dict_neg = {}
for word in tqdm.tqdm(words_neg):
    sim_dict_neg[word] = most_similar_word(word, words_neg, doc_term_mat_neg, vectorizer_neg)

100%|████████████████████████████████████| 10669/10669 [00:45<00:00, 236.84it/s]


In [38]:
# FIND THE BEST CANDIDATE OF CHANGEBALE WORDS IN A SENTENCE 
def change_word_cond(text):
    global sim_dict_1
    global sim_dict_0
    global sim_dict_neg
    
    tokens = word_tokenize(text)
    label = tokens[0]
    sim_vec = []
    #print(label)
    if label == '__label__1':
        for i in tokens:
            flag = True
            if i not in stop_words and i in sim_dict_1.keys():
                for k in range(7):
                    if flag:
                        try: 
                            sim_word = sim_dict_1[i][k]
                            temp = sim_word
                            if rec_diff(i, temp) and tagger.tag(word_tokenize(temp))[0][1] == tagger.tag(word_tokenize(i))[0][1]:
                                flag = False
                                sim_vec.append(temp)
                            elif k==9:
                                    sim_vec.append(' ')
                        except IndexError:
                            pass

            else:
                sim_vec.append(' ')
    
    elif label == '__label__0':
        for i in tokens:
            flag = True
            if i not in stop_words and i in sim_dict_0.keys():
                for k in range(7):
                    if flag:
                        try: 
                            sim_word = sim_dict_0[i][k]
                            temp = sim_word
                            
                            if rec_diff(i, temp) and tagger.tag(word_tokenize(temp))[0][1] == tagger.tag(word_tokenize(i))[0][1]:
                                flag = False
                                sim_vec.append(temp)
                            elif k==9:
                                    sim_vec.append(' ')
                        except IndexError:
                            pass

            else:
                sim_vec.append(' ')

                
    else:
        for i in tokens:
            flag = True
            if i not in stop_words and i in sim_dict_neg.keys():
                for k in range(7):
                    if flag:
                        #try: 
                            sim_word = sim_dict_neg[i][k]
                            temp = sim_word
                            if rec_diff(i, temp) and tagger.tag(word_tokenize(temp))[0][1] == tagger.tag(word_tokenize(i))[0][1]:
                                flag = False
                                sim_vec.append(temp)
                            elif k==9:
                                    sim_vec.append(' ')
                        #except IndexError:
                            #pass

            else:
                sim_vec.append(' ')
    
    
    
    return sim_vec
    

In [39]:
print(change_word_cond(train['sentiment_comment'][4]))
print(change_word(train['sentiment_comment'][4]))
print(train['sentiment_comment'][4])
      

[' ', ' ', ' ', ' ', ' ', ' ', 'گوشت', 'صفها', 'دهیار', 'بن', ' ', 'گوشت', ' ', 'گوشت', ' ', ' ', 'دستمزد', ' ', 'گوشت', 'کارهامو']
[' ', ' ', ' ', ' ', ' ', ' ', 'اسفندماه', 'صفها', 'دراثر', 'اسکان', ' ', 'یخ', ' ', 'یخ', ' ', ' ', 'عمری', ' ', 'برآورم', 'بن']
__label__neg آقای رشید پور به روستای ما بن گوشت یخ زده دادن به شورا و دهیار  دو نفری همه را کشیدن بالا  از توابع قزوین


In [40]:
augmented_list_cond = []
for i in tqdm.tqdm((train['sentiment_comment'])):
    comment = i
    candidate = change_word_cond(comment)
    augmented_list_cond.append(generate_sentence_permutations(candidate, word_tokenize(comment)))
    

100%|█████████████████████████████████████| 5838/5838 [00:05<00:00, 1101.23it/s]


In [41]:

# Initialize an empty list to hold the 1D list
one_d_list_cond = []

# Use nested for loops to iterate over the sublists and items
for sublist in augmented_list_cond:
    for item in sublist:
        one_d_list_cond.append(item)

print(len(one_d_list_cond))


7369


In [42]:
df_conditional = pd.DataFrame(one_d_list_cond)
df_conditional.rename(columns = { 0 :'sentiment_comment', 1: 'numb' }, inplace = True)
df_conditional[df_conditional['sentiment_comment'].str.strip().astype(bool)]

,sentiment_comment
0,__label__neg از محمد اصفهانی دعوت نکنین
1,__label__neg از شاهکارش خودمونم اموزش خورشیدوا...
2,__label__1 سلام با تشکر فراوان از برنامه پر ان...
3,__label__neg رشیدپور از چشم همه افتاده نون به ...
4,__label__0 سالار عقیلی شماره ۶
...,...
7364,__label__neg مسوولان در حال شمردن پولاشونن وقت...
7365,__label__neg چطوری سوپاپ اطمینان کی میخوای خجا...
7366,__label__0 به آقا تنابنده بگید یه تیکه شمالی ح...
7367,__label__0 به استخر رئالی بسازند یه تنابنده جد...


In [43]:
df_conditional['sentiment_comment'].to_csv('cond_aug_TFIDF.train', index=False)

In [44]:
model3 = fasttext.train_supervised("cond_aug_TFIDF.train", dim=300, autotuneValidationFile='t.valid', pretrainedVectors='wiki.fa.vec')

Progress: 100.0% Trials:   11 Best score:  0.759630 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  14907
Number of labels: 3
Progress: 100.0% words/sec/thread:  149268 lr:  0.000000 avg.loss:  0.425270 ETA:   0h 0m 0s


In [45]:
model3.test('t.test')

(721, 0.7226074895977809, 0.7226074895977809)

In [46]:
acc = 0
for i in range(np.array(test).shape[0]):
    if model3.predict(np.array(test['comment'])[i])[0][0] == np.array(test['sentiment'])[i] : 
        acc += 1
print(f'accuracy is {acc/np.array(test).shape[0] *100}%')
  

accuracy is 72.67683772538142%


### perplexity

In [47]:
result2 = [list_[1] for list_ in augmented_list_cond if len(list_) ==2]


In [48]:
generated_sentences = [' '.join(word_tokenize(sent)[1:]) for sent in result2]

In [49]:
model3.predict(generated_sentences[2])

(('__label__1',), array([0.86702371]))

In [50]:
n = sum(len(sentence.split()) for sentence in generated_sentences)
log_probs = [np.log2(model3.predict(sentence, k=1)[1])for sentence in generated_sentences if (model3.predict(sentence, k=1)[1])<1 ]
perplexity = 2 ** (-1/n * sum(log_probs))
print("Perplexity:", perplexity)

Perplexity: [1.0122513]


### diversity

In [51]:
vectors = [model3.get_sentence_vector(sentence) for sentence in generated_sentences]
similarities = cosine_similarity(vectors)
diversity = 1 - similarities.mean()
print("Diversity:", diversity)

Diversity: 0.7162911295890808


### ارزیابی انسانی

In [53]:
res = []
res = [list_ for list_ in augmented_list_cond if len(list_) ==2]
for i in range(10):
    print(f'org{i+1} : {res[i][0]} \n gen{i+1} : {res[i][1]} \n')

org1 : __label__neg از محمد اصفهانی دعوت نکنین 
 gen1 : __label__neg از شاهکارش خودمونم اموزش خورشیدواصلاتموم 

org2 : __label__0 سالار عقیلی شماره ۶ 
 gen2 : __label__0 آقایان وکیلم حسینی ۶ 

org3 : __label__1 یزد خودونم برف أومده إقه خش شده 
 gen3 : __label__1 سیدمحمد خورشیذ برف خورشیذ خورشیذ خورشیذ شده 

org4 : __label__0 مناظره بین دکتر سبطی وآقای بطحایی 
 gen4 : __label__0 آموزش ودکتر طب آقائی بارنگ دکتر 

org5 : __label__1 همه چیز از اوست 
 gen5 : __label__1 همه خوبید از خوبه 

org6 : __label__0 مهران مدیری و خود شما آقای رشید پور عزیز 
 gen6 : __label__0 _علی کریمی و شما شما آقای رشید پور عزیز 

org7 : __label__neg برنامه تون و تمام نکنید لطفا 
 gen7 : __label__neg برنامه تون و تمام وجدانند لطفا 

org8 : __label__0 قسمت سانسور شده چیا بودن 
 gen8 : __label__0 روایت لطف شده خنج سانسور 

org9 : __label__0 همه بازماندگانی که سال NUM۲ سلامت به پایان رساندن 
 gen9 : __label__0 همه سال که سال NUM۲ سال به انتظارمان سال 

org10 : __label__0 محسن از مبا 
 gen10 : __label__0 مبارکی از خمس